<a href="https://colab.research.google.com/github/tumikosha/ds/blob/main/turism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [152]:
# !curl -L -o etapa1_full.csv https://drive.google.com/u/0/uc?id=1_sjviByPoVQPJA5ZDhdOej-HvvifbBXA
# !curl -L -o etapa2.csv https://drive.google.com/u/0/uc?id=1ixzeORjUzJLT7nEetQ6amJP0wQH01DNR
# !curl -L -o etapa3.csv https://drive.google.com/u/0/uc?id=1auvyubEeTFMZpMyzK80HmOkDsbwpUJxJ 
# !curl -L -o etapa4.csv https://drive.google.com/u/0/uc?id=1aBt3zjn6M-ceDpsdXMoZU21q9DbP2z8n 
!curl -L  https://drive.google.com/uc?id=1Hi188_hFHApCCNtCQ6Spv5lO2DgORFUf   >turism.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 1914k  100 1914k    0     0  1378k      0  0:00:01  0:00:01 --:--:-- 1378k


In [ ]:
import pandas as pd
df = pd.read_csv("turism.csv")
df.corr()

In [154]:
import pandas as pd
turism = {
    'VKunitki': {
        'Khalkidhiki': 4.5, 'Thasos': 5.0,  'Crete': 5.0,  'Santorini': 4.5, 'Odessa': 2.5,  'Zatoka': 1.0, 'Romania': 2.0,
        'Krakow': 5.0, 'Rome': 5.0,  'Vienna': 5.0,  'Milano': 5.0,  'Moscow': 5.0,  'Odessa': 2.5, 'Oia': 1,
    },
    'Olga': { 'Khalkidhiki': 4.5,        'Thasos': 5.0,  'Crete': 5.0,        'Santorini': 4.5,   'Odessa': 2.5,    },
    'Sasha': {'Thasos': 5.0,   'Crete': 5.0,  'Santorini': 4.5,  'Odessa': 2.5,   'Vienna': 5.0, 'Berlin': 5.0 , 'Oia': 4 },
    'Ion': {   'Leptokaria': 3.5,   'Moscow': 5.0,    'London': 4.5,  }
}


In [155]:
def sim_distance(prefs, p1, p2):
    '''
    Returns a distance-based similarity score for person1 and person2.
    '''

    # Get the list of shared_items
    si = {}
    for item in prefs[p1]:
        if item in prefs[p2]:
            si[item] = 1
    # If they have no ratings in common, return 0
    if len(si) == 0:
        return 0
    # Add up the squares of all the differences
    sum_of_squares = sum([pow(prefs[p1][item] - prefs[p2][item], 2) for item in
                         prefs[p1] if item in prefs[p2]])
    return 1 / (1 + sqrt(sum_of_squares))


def sim_pearson(prefs, p1, p2):
    '''
    Returns the Pearson correlation coefficient for p1 and p2.
    '''

    # Get the list of mutually rated items
    si = {}
    for item in prefs[p1]:
        if item in prefs[p2]:
            si[item] = 1
    # If they are no ratings in common, return 0
    if len(si) == 0:
        return 0
    # Sum calculations
    n = len(si)
    # Sums of all the preferences
    sum1 = sum([prefs[p1][it] for it in si])
    sum2 = sum([prefs[p2][it] for it in si])
    # Sums of the squares
    sum1Sq = sum([pow(prefs[p1][it], 2) for it in si])
    sum2Sq = sum([pow(prefs[p2][it], 2) for it in si])
    # Sum of the products
    pSum = sum([prefs[p1][it] * prefs[p2][it] for it in si])
    # Calculate r (Pearson score)
    num = pSum - sum1 * sum2 / n
    den = sqrt((sum1Sq - pow(sum1, 2) / n) * (sum2Sq - pow(sum2, 2) / n))
    if den == 0:
        return 0
    r = num / den
    return r


def topMatches(
    prefs,
    person,
    n=5,
    similarity=sim_pearson,
):
    '''
    Returns the best matches for person from the prefs dictionary.
    Number of results and similarity function are optional params.
    '''

    scores = [(similarity(prefs, person, other), other) for other in prefs
              if (other != person) and (type(other) is IntType) ]
    for other in prefs:
         print(other, type(other))
         print(float(other))
    print(scores)
    scores.sort() # TypeError: unorderable types: str() < int()
    scores.reverse()
    return scores[0:n]


def getRecommendations(prefs, person, similarity=sim_pearson):
    # Получить рекомендации для заданного человека, пользуясь взвешенным средним
    # оценок, данных всеми остальными пользователями

    totals = {}
    simSums = {}
    for other in prefs:
        # сравнивать меня с собой же не нужно
        if other == person: continue
        sim = similarity(prefs, person, other)
        # игнорировать нулевые и отрицательные оценки
        if sim <= 0:
            continue
        for item in prefs[other]:
            # оценивать только фильмы, которые я еще не смотрел
            if item not in prefs[person] or prefs[person][item] == 0:
                ## Коэффициент подобия * Оценка = Similarity * Score
                totals.setdefault(item, 0)
                # The final score is calculated by multiplying each item by the
                #   similarity and adding these products together
                totals[item] += prefs[other][item] * sim
                ## Сумма коэффициентов подобия,  Sum of similarities
                simSums.setdefault(item, 0)
                simSums[item] += sim
    # Создать нормализованный список
    rankings = [(total / simSums[item], item) for (item, total) in
                totals.items()]
    # Вернуть отсортированный список, # Return the sorted list
    rankings.sort()
    rankings.reverse()
    return rankings


In [157]:

import pandas as pd
df = pd.read_csv("turism.csv")
# transforming pandas into trips structure
turism = {
    'Ion': {   'Leptokaria': 3.5,   'Moscow': 5.0,    'London': 4.5,  }
}

def nota(txt):
  txt = txt.lower().strip()
  if txt == "very satisfied": return 5
  if txt == "satisfied": return 4
  if txt == "neither satisfied, nor dissatisfied": return 3
  if txt == "dissatisfied": return 2
  if txt == "very dissatisfied": return 1
  return 0

def code(v1, v2, obj):
    if v1=="I visited":  
        if nota(v2)!=0 :  trips[obj]= nota(v2) 

# for idx, row in df[0:20].iterrows():
for idx, row in df.iterrows():
    # print(row['id'])
    trips={} 
    code(row['Q26_11'], row['Q27_11'], 'Museums')  
    code(row['Q26_12'], row['Q27_12'], 'Theaters')  

    code(row['Q26_13'], row['Q27_13'], 'Restaurants') 
    code(row['Q26_13'], row['Q27_13'], 'Wine')  
    code(row['Q26_13'], row['Q27_13'], 'Clubs') 

    code(row['Q26_21'], row['Q27_21'], 'Milestii mici')  
    code(row['Q26_22'], row['Q27_22'], 'Cricova')         
    code(row['Q26_23'], row['Q27_23'], 'Purcari')         
    code(row['Q26_24'], row['Q27_24'], 'Chateau Cojushna')
    code(row['Q26_25'], row['Q27_25'], 'Chateau Vartely')
    code(row['Q26_26'], row['Q27_26'], 'Castel Mimi')
    code(row['Q26_27'], row['Q27_27'], 'Asconi Winery')
    code(row['Q26_27'], row['Q27_27'], 'EtCetera Winery')
    code(row['Q26_3'], row['Q27_3'], 'Orheiul Vechi')
    code(row['Q26_41'], row['Q27_41'], 'Capriana')
    code(row['Q26_42'], row['Q27_42'], 'Saharna')
    code(row['Q26_43'], row['Q27_43'], 'Tipova')  

    code(row['Q26_51'], row['Q27_51'], 'Soroca fortress')  
    code(row['Q26_52'], row['Q27_52'], 'The castle Manuc Bei') 
    code(row['Q26_53'], row['Q27_53'], 'Castle Mimi') 
    
    code(row['Q26_61'], row['Q27_61'], 'Padurea Domneasca') 
    code(row['Q26_62'], row['Q27_62'], 'Codri') 
    code(row['Q26_63'], row['Q27_63'], 'Cave Emil Racovita') 
    code(row['Q26_64'], row['Q27_64'], 'Lower Prut Biosphere') 

    code(row['Q26_71'], row['Q27_71'], 'Chisinau nternational Marathon (September)') 
    code(row['Q26_72'], row['Q27_72'], 'Ethnojazz Festival Spt. 20-30') 
    code(row['Q26_73'], row['Q27_73'], 'Wine Festival Oct 6-7') 
    code(row['Q26_74'], row['Q27_74'], 'City Anniversary') 
    code(row['Q26_74'], row['Q27_74'], 'Weddings/cumatria')     

    code(row['Q26_81'], row['Q27_81'], 'Weddings/cumatria')     
    code(row['Q26_82'], row['Q27_82'], 'Casa din Lunca')  
    code(row['Q26_83'], row['Q27_83'], 'Hanu lui Ganganu') 
    code(row['Q26_84'], row['Q27_84'], 'Fata Morgana') 
    code(row['Q26_85'], row['Q27_85'], 'The greeen house/Casa Verde') 
    code(row['Q26_86'], row['Q27_86'], 'Pink Villa/Vila Roz') 
    # print(idx, trips)
    turism[row['id']]=trips

my_name = "Newbie"
turism[my_name] = {'Cricova':1, 'Purcari':1}   
print("recommendations: ", getRecommendations(turism, my_name, similarity=sim_distance))


recommendations:  [(4.776527513235389, 'EtCetera Winery'), (4.776527513235389, 'Asconi Winery'), (4.762992047566224, 'Milestii mici'), (4.759241468071656, 'Castle Mimi'), (4.722337080232387, 'Wine Festival Oct 6-7'), (4.718871873386073, 'Chateau Vartely'), (4.710556558469732, 'Castel Mimi'), (4.702087619943356, 'City Anniversary'), (4.687033185196309, 'Orheiul Vechi'), (4.643624476277735, 'Weddings/cumatria'), (4.638962104383786, 'Theaters'), (4.6344872013297325, 'Chateau Cojushna'), (4.631969957271742, 'Wine'), (4.631969957271742, 'Restaurants'), (4.631969957271742, 'Clubs'), (4.623012422845682, 'Saharna'), (4.591811589093858, 'Lower Prut Biosphere'), (4.591677890340191, 'Casa din Lunca'), (4.591108432671799, 'Tipova'), (4.589958142054848, 'Soroca fortress'), (4.568934813547271, 'Capriana'), (4.5549757228838565, 'Fata Morgana'), (4.507654598993962, 'Museums'), (4.498399346695305, 'Hanu lui Ganganu'), (4.488888888677119, 'Padurea Domneasca'), (4.4519544122823, 'The greeen house/Casa Ve